In [48]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
from sklearn import model_selection
from IPython.display import display, HTML
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph
from stellargraph import datasets
import networkx as nx
from stellargraph.data import BiasedRandomWalk

In [49]:
LIMIT = 10

In [50]:
bitcoin_address =  pd.read_csv('bitcoin_address.csv',nrows=LIMIT)

bitcoin_address_link =  pd.read_csv('bitcoin_address_link.csv',nrows=LIMIT)

page =  pd.read_csv('page.csv',nrows=LIMIT)

page_link =  pd.read_csv('page_link.csv',nrows=LIMIT)

domain =  pd.read_csv('domain.csv',nrows=LIMIT)

In [51]:
G1 = nx.Graph()
node_subjects = {}

#insert bitcoin address nodes

for index, row in tqdm(bitcoin_address.iterrows()):
    G1.add_node(row.id)
    node_subjects[row.id]='b'
    
#insert pages

for index, row in tqdm(page.iterrows()):
    G1.add_node(row.id)
    node_subjects[row.id]='p'
    
#insert page - bitcoin addres edges

for index, row in tqdm(bitcoin_address_link.iterrows()):
    G1.add_edge(row.bitcoin_address, row.page)
    
#insert page - page edges

for index, row in tqdm(page_link.iterrows()):
    G1.add_edge(row.link_from, row.link_to )
    
    
#insert domain

for index, row in tqdm(domain.iterrows()):
    G1.add_node(row.id)
    node_subjects.append('d')
    
#insert page - domain edges

for index, row in tqdm(page.iterrows()):
    G1.add_edge(row.id, row.domain )


In [52]:
g1 =  StellarGraph.from_networkx(G1)
print(g1.info())

StellarGraph: Undirected multigraph
 Nodes: 48, Edges: 30

 Node types:
  default: [48]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [30]
        Weights: all 1 (default)
        Features: none


In [53]:
#Corpus generation using random walks

rw = BiasedRandomWalk(g1)

walks = rw.run(
    nodes=list(g1.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

Number of random walks: 480


In [54]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=1)

In [55]:
# The embedding vectors can be retrieved from model.wv using the node ID as key.
# list(model.wv)

In [56]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index2word  # list of node IDs
node_embeddings = (
    model.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = node_subjects[[int(node_id) for node_id in node_ids]]

TypeError: list indices must be integers or slices, not list

In [57]:
temp = [int(node_id) for node_id in node_ids]
[node_subjects[i] for i in temp]

IndexError: list index out of range

In [ ]:
# Apply t-SNE transformation on node embeddings
tsne = TSNE(n_components=2)
node_embeddings_2d = tsne.fit_transform(node_embeddings)

In [ ]:
# draw the points
alpha = 0.7
label_map = {l: i for i, l in enumerate(np.unique(node_targets))}
node_colours = [label_map[target] for target in node_targets]

plt.figure(figsize=(10, 8))
plt.scatter(
    node_embeddings_2d[:, 0],
    node_embeddings_2d[:, 1],
    c=node_colours,
    cmap="jet",
    alpha=alpha,
)

In [ ]:
#Downstream task
# X will hold the 128-dimensional input features
X = node_embeddings
# y holds the corresponding target values
y = np.array(node_targets)

In [ ]:
#Data Splitting

In [16]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, node_subjects = dataset.load(largest_connected_component_only=True)

In [19]:
node_subjects

31336             Neural_Networks
1061127             Rule_Learning
1106406    Reinforcement_Learning
13195      Reinforcement_Learning
37879       Probabilistic_Methods
                    ...          
1128975        Genetic_Algorithms
1128977        Genetic_Algorithms
1128978        Genetic_Algorithms
117328                 Case_Based
24043             Neural_Networks
Name: subject, Length: 2485, dtype: object